### Creating and Persisting an ML Model

In [183]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/student-mat.csv', sep=';')

Summary of the data

In [184]:
df.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,2.749367,2.521519,1.448101,2.035443,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861,10.908861,10.713924,10.415190
std,1.276043,1.094735,1.088201,0.697505,0.839240,0.743651,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096,3.319195,3.761505,4.581443
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,8.000000,9.000000,8.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,11.000000,11.000000,11.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,13.000000,13.000000,14.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,20.000000


In [185]:
df.info

<bound method DataFrame.info of     school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
390     MS   M   20       U     LE3       A     2     2  services  services   
391     MS   M   17       U     LE3       T     3     1  services  services   
392     MS   M   21       R     GT3       T     1     1     other     other   
393     MS   M   18       R     LE3       T     3     2  services     other   
394     MS   M   19       U     LE3       T     1     1     other   at_home   

     ... famrel fre

Create a subset of features as an example.

In [186]:
include = ['health', 'absences', 'freetime','famrel','studytime','failures','G3']
df.drop(columns=df.columns.difference(include), inplace=True)  # only using 3 features

In [187]:
df.info

<bound method DataFrame.info of      studytime  failures  famrel  freetime  health  absences  G3
0            2         0       4         3       3         6   6
1            2         0       5         3       3         4   6
2            2         3       4         3       3        10  10
3            3         0       3         2       5         2  15
4            2         0       4         3       5         4  10
..         ...       ...     ...       ...     ...       ...  ..
390          2         2       5         5       4        11   9
391          1         0       2         4       2         3  16
392          1         3       5         5       3         3   7
393          1         0       4         4       5         0  10
394          1         0       3         2       5         5   9

[395 rows x 7 columns]>

The goal is to predict the quality of the student. We will build a predictor based on the final grade (G3).
Becasue we are trying to find quality students. In this model we define a quality student as one who achieves a final grade of 15 or higher. 

In [188]:
df['qual_student'] = np.where(df['G3']>=15, 1, 0)

In [189]:
df.describe()

,studytime,failures,famrel,freetime,health,absences,G3,qual_student
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,2.035443,0.334177,3.944304,3.235443,3.554430,5.708861,10.415190,0.184810
std,0.839240,0.743651,0.896659,0.998862,1.390303,8.003096,4.581443,0.388636
min,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,4.000000,3.000000,3.000000,0.000000,8.000000,0.000000
50%,2.000000,0.000000,4.000000,3.000000,4.000000,4.000000,11.000000,0.000000
75%,2.000000,0.000000,5.000000,4.000000,5.000000,8.000000,14.000000,0.000000
max,4.000000,3.000000,5.000000,5.000000,5.000000,75.000000,20.000000,1.000000


Drop the G3 score

In [190]:
include1 = ['health', 'absences', 'freetime','studytime','failures','schoolsup','famsup','Dalc','Walc','qual_student']
include2 = ['health', 'absences', 'age', 'qual_student']
df1 = df
df2 = df
df1.drop(columns=df.columns.difference(include1), inplace=True) 
df2.drop(columns=df.columns.difference(include2), inplace=True) 

Import scikit-learn and build a random forest classifer

In [191]:
from sklearn.ensemble import RandomForestClassifier as rf
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

dependent_variable = 'qual_student'
x1 = df1[df.columns.difference([dependent_variable])]
y1 = df1[dependent_variable]
x2 = df2[df.columns.difference([dependent_variable])]
y2 = df2[dependent_variable]

xTrain, xTest, yTrain, yTest = train_test_split(x1, y1, test_size=0.2)

#our model
from sklearn.ensemble import AdaBoostClassifier
newclf = AdaBoostClassifier()
newclf.fit(xTrain, yTrain)

#baseline model
baseclf = rf(n_estimators=1000)
baseclf.fit(xTrain, yTrain)

RandomForestClassifier(n_estimators=1000)

In [192]:
print('Baseline Model Report')
pred = baseclf.predict(xTrain)
print('#Train Set Accuracy: ' + str(accuracy_score(yTrain, pred)))
pred = baseclf.predict(xTest)
print('#Test Set Accuracy: ' + str(accuracy_score(yTest, pred)))
print('')
print('New Model Report')
pred = newclf.predict(xTrain)
print('#Train Set Accuracy: ' + str(accuracy_score(yTrain, pred)))
pred = newclf.predict(xTest)
print('#Test Set Accuracy: ' + str(accuracy_score(yTest, pred)))


Baseline Model Report
#Train Set Accuracy: 0.8575949367088608
#Test Set Accuracy: 0.7341772151898734

New Model Report
#Train Set Accuracy: 0.8354430379746836
#Test Set Accuracy: 0.759493670886076


In [193]:
import joblib
# modify the file path to where you want to save the model
joblib.dump(clf1, 'dockerfile/apps/model.pkl')

['dockerfile/apps/model.pkl']